In [26]:
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split

In [36]:
WINSIZE = 101   # for this dataset

class NursingDatasetV1(Dataset):

    def __init__(self, dir: Path, session_idxs: list) -> None:
        super().__init__()
        self.dir = dir

    def __getitem__(self, index: int) -> torch.Tensor:
        pass

    def __len__(self) -> int:
        # Total number of windows is length of dataset
        return sum(self._lengths)
    
    @staticmethod
    def nursingv1_train_dev_test_split(dir: Path, train_size: float, dev_size: float, test_size: float) -> tuple:
        # Returns tuple of NursingDatasetV1 objects
        # Each of the objects is given a subset of total sessions corrosponding
        #   to the parameters

        # Get list of all session idxs in dataset and their lengths (# of windows)
        sessions = []
        lengths = []

        for session in dir.iterdir():
            sessions.append(session.name)
            session_shape = torch.load(session / 'Xshape.pt')

            # Save number of windows, which is session length - winsize + 1
            lengths.append(session_shape[1] - WINSIZE + 1)
        
        # Split sessions into train, dev, and test
        train_idxs, dev_test_idxs = train_test_split(sessions, train_size=train_size)
        dev_idxs, test_idxs = train_test_split(dev_test_idxs, train_size=dev_size)  # TODO fix - dev_size is % of subset not entire set now

        print(len(train_idxs))
        print(len(dev_idxs))
        print(len(test_idxs))


In [37]:
nursingv1_dir = Path('../data/nursingv1_dataset')
# nursing_v1 = NursingDatasetV1(nursingv1_dir, 0.5, 0.2, 0.3)
NursingDatasetV1.nursingv1_train_dev_test_split(nursingv1_dir, 0.5, 0.3, 0.2)

29
8
21
